# Running Palace Simulations: Inductor

[Palace](https://awslabs.github.io/palace/) is an open-source 3D electromagnetic simulator supporting eigenmode, driven (S-parameter), and electrostatic simulations. This notebook demonstrates using the `gsim.palace` API to run a driven simulation on an on-chip spiral inductor.

**Requirements:**

- IHP PDK: `uv pip install ihp-gdsfactory`
- [GDSFactory+](https://gdsfactory.com) account for cloud simulation

### Load inductor pcell from IHP PDK

In [ ]:
import gdsfactory as gf
from ihp import LAYER, PDK, cells

PDK.activate()

c = gf.Component()

# Add spiral inductor (1 turn, ~33pH)
ind = c << cells.inductor2(turns=2, diameter=40, width=2.0, space=2.1)

# Add Metal1 ground plane underneath the inductor
bbox = ind.dbbox()
ground_margin = 10  # um margin around inductor
c.add_polygon(
    [
        (bbox.left - ground_margin, bbox.bottom - ground_margin),
        (bbox.right + ground_margin, bbox.bottom - ground_margin),
        (bbox.right + ground_margin, bbox.top + ground_margin),
        (bbox.left - ground_margin, bbox.top + ground_margin),
    ],
    layer=LAYER.Metal1drawing,
)

# Copy ports from inductor
c.add_ports(ind.ports)

cc = c.copy()
cc.draw_ports()
cc

### Configure and run simulation with DrivenSim

In [ ]:
from gsim.palace import DrivenSim

# Create simulation object
sim = DrivenSim()

# Set output directory
sim.set_output_dir("./palace-sim-inductor")

# Set the component geometry
sim.set_geometry(c)

# Configure layer stack from active PDK
# substrate_thickness: silicon substrate depth (um)
# air_above: air box above top metal (um)
sim.set_stack(substrate_thickness=200.0, air_above=100.0)

# Configure via ports (Metal1 ground plane to TopMetal2 signal)
for port in c.ports:
    sim.add_port(port.name, from_layer="metal1", to_layer="topmetal2", geometry="via")

# Configure driven simulation (frequency sweep for S-parameters)
sim.set_driven(fmin=1e9, fmax=50e9, num_points=50)

# Validate configuration
print(sim.validate_config())

In [ ]:
# Generate mesh (presets: "coarse", "default", "fine")
sim.mesh(preset="default")

In [ ]:
# Static PNG
# sim.plot_mesh(show_groups=["metal", "P"], interactive=False)

# Interactive
sim.plot_mesh(show_groups=["metal", "P"], interactive=True)

In [ ]:
dfsdf

In [ ]:
# Generate Palace config file
sim.write_config()

### Run simulation on GDSFactory+ Cloud

In [ ]:
# Run simulation on GDSFactory+ cloud
results = sim.simulate()

In [ ]:
import os

print(os.environ.get("GFP_API_HOST"))

### Plot S-parameters

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(results["port-S.csv"])
df.columns = df.columns.str.strip()  # Remove whitespace from column names

freq = df["f (GHz)"]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

# Magnitude plot
ax1.plot(freq, df["|S[1][1]| (dB)"], marker=".", label="S11")
ax1.plot(freq, df["|S[2][1]| (dB)"], marker=".", label="S21")
ax1.set_xlabel("Frequency (GHz)")
ax1.set_ylabel("Magnitude (dB)")
ax1.set_title("Inductor S-Parameters")
ax1.legend()
ax1.grid(True)

# Phase plot
ax2.plot(freq, df["arg(S[1][1]) (deg.)"], marker=".", label="S11")
ax2.plot(freq, df["arg(S[2][1]) (deg.)"], marker=".", label="S21")
ax2.set_xlabel("Frequency (GHz)")
ax2.set_ylabel("Phase (deg)")
ax2.legend()
ax2.grid(True)

plt.tight_layout()

### Extract Inductance from S-parameters

For a series inductor, we can extract the inductance from the S-parameters using:

$$Z = Z_0 \frac{1 + S_{11}}{1 - S_{11}}$$

$$L = \frac{\text{Im}(Z)}{2\pi f}$$

In [ ]:
import numpy as np

# Convert S11 magnitude and phase to complex S11
s11_mag_db = df["|S[1][1]| (dB)"].values
s11_phase_deg = df["arg(S[1][1]) (deg.)"].values
s11_mag = 10 ** (s11_mag_db / 20)
s11 = s11_mag * np.exp(1j * np.deg2rad(s11_phase_deg))

# Calculate impedance
Z0 = 50  # Reference impedance
Z = Z0 * (1 + s11) / (1 - s11)

# Extract inductance
freq_hz = freq.values * 1e9
L = np.imag(Z) / (2 * np.pi * freq_hz)

# Plot inductance vs frequency
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

ax1.plot(freq, L * 1e12, marker=".")
ax1.set_xlabel("Frequency (GHz)")
ax1.set_ylabel("Inductance (pH)")
ax1.set_title("Extracted Inductance")
ax1.grid(True)
ax1.set_ylim(bottom=0)

# Q factor: Q = Im(Z) / Re(Z)
Q = np.imag(Z) / np.real(Z)
ax2.plot(freq, Q, marker=".")
ax2.set_xlabel("Frequency (GHz)")
ax2.set_ylabel("Quality Factor (Q)")
ax2.set_title("Inductor Q Factor")
ax2.grid(True)
ax2.set_ylim(bottom=0)

plt.tight_layout()

# Print average inductance in low-frequency region
low_freq_mask = freq < 10  # Below 10 GHz
L_avg = np.mean(L[low_freq_mask]) * 1e12
print(f"Average inductance (1-10 GHz): {L_avg:.1f} pH")